### Installations

In [1]:
#!pip install lxml
#!pip install wget
#!pip install gpxpy

### Imports

In [2]:
# import requests
import re
from bs4 import BeautifulSoup 
import pandas as pd
import lxml

import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import requests
#import wget
import glob
import datetime as dt
import time
import seaborn as sns
from matplotlib import rcParams
import matplotlib.pyplot as plt

import time
import gpxpy 
import gpxpy.gpx 
import glob

# Still to do:
- Add key words: Webscraping from a static page, Data Cleaning, plots, plotly?, GPX coordinates, KML coordinates (add both? describe the differences?), spatial data, tableau, visualization

- import csv file instead of rescrapping (hash out the rest)

- DONE: change difficulty to '0 - No rating', '1 - Easy' etc
- remove '0 - No rating' for some plots
- DONE: easy = green etc
- nicer look of plots?

- simplify gpx code
- run gpx code for all 257 codes (check if not blocked)

- check why datetime code does not always work

- Check comment with Google earth and "KML" file

# 1  Data scraping

Prepare to scrape the Bergfex.com website in order to create Pandas dataframes that contain important information about each trail. This code uses BeautifulSoup to parse the html tags into json, which is easier to work with. A for-loop iterates over each html tag and adds the corresponding information into empty lists. 

After running the code in this section "Data scraping" once, we recommend hashing out the code to avoid being blocked by the website for sending too many requests.

### 1.1 Scrape all the information (basic and more complicated)

In [3]:
# remove the """""" to run the code

"""title = []  # Title of the tour
difficulty = []  # difficulty (easy, medium, hard)
sport = []  # Sport type (hiking, sledging, snowshoe...)
length = []  # length in km
time_list = []  # tour time in hours:minutes
climb = []  # positive elevation climb in m
minmax = []  # minimum and maximum altitude of the tour in m
technique = []  # technique difficulty ratings (out of 6)
fitness = []  # fitness difficulty ratings (out of 6)
total_title = [] # will help extracting the ID of the tour

# all ratings are stored together so we will need this along the way:
rating = []  # list to store technique and fitness rating info"""

'title = []  # Title of the tour\ndifficulty = []  # difficulty (easy, medium, hard)\nsport = []  # Sport type (hiking, sledging, snowshoe...)\nlength = []  # length in km\ntime_list = []  # tour time in hours:minutes\nclimb = []  # positive elevation climb in m\nminmax = []  # minimum and maximum altitude of the tour in m\ntechnique = []  # technique difficulty ratings (out of 6)\nfitness = []  # fitness difficulty ratings (out of 6)\ntotal_title = [] # will help extracting the ID of the tour\n\n# all ratings are stored together so we will need this along the way:\nrating = []  # list to store technique and fitness rating info'

In [4]:
# looping over the x first pages of Bergfex
# each separate tour on the page is framed by a div tag with 'touren-details'

""""page_number = 20  # number of pages we want to scap through

for p in range(1, (page_number+1)):
    base_link = 'https://www.bergfex.com/sommer/bern-region/touren/?isAjax=1&page='
    link = base_link+str(p)  # going over p pages with numbers appended to the base link
    page = requests.get(link, timeout=5)
    print("scraped page", p)
    soup = BeautifulSoup(page.content, "html.parser")  # bs4.BeautifulSoup object
    tours = soup.findAll('div', {'class': 'touren-detail'})  # checks for the separate tours on the page

# For each page, we loop over each tour and fill our lists with info     
    for i in range(0,len(tours)):
        tour_1 = tours[i] # tour iterating over the 20 tours of the page

        tour_title = tour_1.findAll('a' ) # this gets the title out
        title.append([info.get_text().strip() for info in tour_title])
        total_title.append(tour_title) 
        # this gets the full tour-title information including the ID, will need to be processed further later on
        
        tour_diff = tour_1.findAll('span', {'class': 'tour-difficulty'}) #tour level difficulty
        difficulty.append([info.get_text().strip() for info in tour_diff])

        tour_type = tour_1.findAll('span', {'class': 'tour-type'}) # putting type into sports
        sport.append([info.get_text().strip() for info in tour_type])

        tour_stats = tour_1.findAll('div', {'class': 'tour-stats'}) # stats has 4 info binned together
        stat_text = [info.get_text().strip() for info in tour_stats]
        length.append(stat_text[0])
        time_list.append(stat_text[1])
        climb.append(stat_text[2])
        minmax.append(stat_text[3])
        
        tour_rating = tour_1.find_all("div", {'class': 'tour-rating'}) # getting the rating data 
        rating.append([info for info in tour_rating])   # it's a class name so we can't get_text."""

'"page_number = 20  # number of pages we want to scap through\n\nfor p in range(1, (page_number+1)):\n    base_link = \'https://www.bergfex.com/sommer/bern-region/touren/?isAjax=1&page=\'\n    link = base_link+str(p)  # going over p pages with numbers appended to the base link\n    page = requests.get(link, timeout=5)\n    print("scraped page", p)\n    soup = BeautifulSoup(page.content, "html.parser")  # bs4.BeautifulSoup object\n    tours = soup.findAll(\'div\', {\'class\': \'touren-detail\'})  # checks for the separate tours on the page\n\n# For each page, we loop over each tour and fill our lists with info     \n    for i in range(0,len(tours)):\n        tour_1 = tours[i] # tour iterating over the 20 tours of the page\n\n        tour_title = tour_1.findAll(\'a\' ) # this gets the title out\n        title.append([info.get_text().strip() for info in tour_title])\n        total_title.append(tour_title) \n        # this gets the full tour-title information including the ID, will need to

In [5]:
"""print("number of tours collected:", len(title))"""

'print("number of tours collected:", len(title))'

### 1.2 Extracting embedded HTML tags: <a href="" 
Each trail name and ID was used in Bergfex's URL. Using the trail ID in the dataframe will provide a unique ID to use later in the analysis. However, those two data points are embedded deeper in the html code. The below codes shows how to extract the datapoints into a dataframe.

In [6]:
# Extracting the ID via the URL
"""links_title = []

for i in total_title:
    i = str(i)
    i = i.strip('[]')
    links_title.append(i)"""

"links_title = []\n\nfor i in total_title:\n    i = str(i)\n    i = i.strip('[]')\n    links_title.append(i)"

In order to make the trail ID usable it needs to be cleaned up and converted into a string

In [7]:
# Extracting ID
"""id_df3 = pd.DataFrame(links_title)
id_df2 = id_df3[0].str.split(' ',expand = True)
id_df1 = id_df2[2].str.split(',',expand = True)
id_df = id_df1[0].str.split('/',expand = True)"""

"id_df3 = pd.DataFrame(links_title)\nid_df2 = id_df3[0].str.split(' ',expand = True)\nid_df1 = id_df2[2].str.split(',',expand = True)\nid_df = id_df1[0].str.split('/',expand = True)"

In [8]:
# from this format:
"""pd.options.display.max_colwidth = 100
id_df3.head()"""

'pd.options.display.max_colwidth = 100\nid_df3.head()'

In [9]:
# to this format:
"""id_df.head()"""

'id_df.head()'

### 1.3 Extracting embedded HTML tags: <Div class=""
The HTML tags for the fitness and technique ratings were embedded within Div class tags; they are extracted using a for loop and then stripped of unecessary characeters. 
- Convert the tour rating tag into a string
- Strip away unecessary characters 
- Split into lists which look like rating_list[Technique, Fitness, Technique, Fitness]
- Finally, convert into seperate lists for Technique and Fitness

In [10]:
"""rating[0] # View the emmbedded DIV tag"""

'rating[0] # View the emmbedded DIV tag'

In [11]:
"""# Replace unwanted information
tour_rating_str = str(rating)
rating_all_short = tour_rating_str.replace(
'<div class="tour-rating">\n<div class="tour-rating-label">Technique</div>\n<div class="rating-circles rating-max6"><div class="',''
).replace(
    '<div class="tour-rating">\n<div class="tour-rating-label">Fitness</div>\n<div class="rating-circles rating-max6"><div class="','').replace('"></div></div>\n</div>',"")
rating_even_shorter = rating_all_short.replace('[','').replace(']','')
rating_list = rating_even_shorter.split(", ")

for i in range(0,(len(rating_list))):
    if i == 0:
        technique.append(rating_list[i]) # add first item in technique
    elif i % 2 == 0:
        technique.append(rating_list[i]) # then every second item as well
    else:
        fitness.append(rating_list[i]) # the other appended to fitness list"""

'# Replace unwanted information\ntour_rating_str = str(rating)\nrating_all_short = tour_rating_str.replace(\n\'<div class="tour-rating">\n<div class="tour-rating-label">Technique</div>\n<div class="rating-circles rating-max6"><div class="\',\'\'\n).replace(\n    \'<div class="tour-rating">\n<div class="tour-rating-label">Fitness</div>\n<div class="rating-circles rating-max6"><div class="\',\'\').replace(\'"></div></div>\n</div>\',"")\nrating_even_shorter = rating_all_short.replace(\'[\',\'\').replace(\']\',\'\')\nrating_list = rating_even_shorter.split(", ")\n\nfor i in range(0,(len(rating_list))):\n    if i == 0:\n        technique.append(rating_list[i]) # add first item in technique\n    elif i % 2 == 0:\n        technique.append(rating_list[i]) # then every second item as well\n    else:\n        fitness.append(rating_list[i]) # the other appended to fitness list'

In [12]:
"""## Let's look at the result

#rating_all_short
#rating_even_shorter
rating_list"""

"## Let's look at the result\n\n#rating_all_short\n#rating_even_shorter\nrating_list"

### 1.4  Create dataframe for extracted elements

This dataframe is not in the final stage yet as there are still extra brackets, the minimum and maximum elevation are still in one column, and units must be stripped away

In [13]:
"""activities_df_unclean = pd.DataFrame(
    {'title': title,
     'difficulty': difficulty,
     'sport': sport,
     'length': length,
     'time': time_list,
     'climb': climb,
     'minmax': minmax,
     'technique': technique,
     'fitness': fitness
    })
activities_df_unclean['ID']= id_df.iloc[:,5].copy() # add the ID columns from an earlier DF
activities_df_unclean"""

"activities_df_unclean = pd.DataFrame(\n    {'title': title,\n     'difficulty': difficulty,\n     'sport': sport,\n     'length': length,\n     'time': time_list,\n     'climb': climb,\n     'minmax': minmax,\n     'technique': technique,\n     'fitness': fitness\n    })\nactivities_df_unclean['ID']= id_df.iloc[:,5].copy() # add the ID columns from an earlier DF\nactivities_df_unclean"

### 1.5 Write CSV file of scrapped and uncleaned DF

In [14]:
"""activities_df_unclean.to_csv("../data/unclean_activities.csv", index=False)"""

'activities_df_unclean.to_csv("../data/unclean_activities.csv", index=False)'

###  1.6 Combine all in scraping function

In [34]:
def scraping_region(region, n_pages):
    title = []  # Title of the tour
    difficulty = []  # difficulty (easy, medium, hard)
    sport = []  # Sport type (hiking, sledging, snowshoe...)
    length = []  # length in km
    time_list = []  # tour time in hours:minutes
    climb = []  # positive elevation climb in m
    minmax = []  # minimum and maximum altitude of the tour in m
    total_title = [] # will help extracting the ID of the tour
    # all ratings are stored together so we will need this along the way:
    rating = []  # list to store technique and fitness rating info
    
    # looping over the x first pages of Bergfex
    # each separate tour on the page is framed by a div tag with 'touren-details'
    page_number = n_pages  # number of pages we want to scap through

    for p in range(1, (page_number+1)):
        base_link = f'https://www.bergfex.com/sommer/{region}/touren/?isAjax=1&page='
        link = base_link+str(p)  # going over p pages with numbers appended to the base link
        page = requests.get(link, timeout=5)
        print("scraped page", p)
        soup = BeautifulSoup(page.content, "html.parser")  # bs4.BeautifulSoup object
        tours = soup.findAll('div', {'class': 'touren-detail'})  # checks for the separate tours on the page

    # For each page, we loop over each tour and fill our lists with info     
        for i in range(0,len(tours)):
            tour_1 = tours[i] # tour iterating over the 20 tours of the page

            tour_title = tour_1.findAll('a' ) # this gets the title out
            title.append([info.get_text().strip() for info in tour_title])
            total_title.append(tour_title) 
            # this gets the full tour-title information including the ID, processed further later on

            tour_diff = tour_1.findAll('span', {'class': 'tour-difficulty'}) #tour level difficulty
            difficulty.append([info.get_text().strip() for info in tour_diff])

            tour_type = tour_1.findAll('span', {'class': 'tour-type'}) # putting type into sports
            sport.append([info.get_text().strip() for info in tour_type])

            tour_stats = tour_1.findAll('div', {'class': 'tour-stats'}) # stats has 4 info binned together
            stat_text = [info.get_text().strip() for info in tour_stats]
            length.append(stat_text[0])
            time_list.append(stat_text[1])
            climb.append(stat_text[2])
            minmax.append(stat_text[3])

            tour_rating = tour_1.find_all("div", {'class': 'tour-rating'}) # getting the rating data 
            rating.append([info for info in tour_rating])   # it's a class name so we can't get_text.
    print("number of tours collected:", len(title))
    return (title, difficulty, sport, length, time_list, climb, minmax, total_title, rating)

In [18]:
def extracting_id(total_title):
    
    links_title = []
    for i in total_title:
        i = str(i)
        i = i.strip('[]')
        links_title.append(i)
    # Extracting ID
    id_df3 = pd.DataFrame(links_title)
    id_df2 = id_df3[0].str.split(' ',expand = True)
    id_df1 = id_df2[2].str.split(',',expand = True)
    id_df = id_df1[0].str.split('/',expand = True)
    
    return id_df

In [29]:
def extracting_rating(rating):
    technique = []  # technique difficulty ratings (out of 6)
    fitness = []  # fitness difficulty ratings (out of 6)
    # Replace unwanted information
    tour_rating_str = str(rating)
    rating_all_short = tour_rating_str.replace(
    '<div class="tour-rating">\n<div class="tour-rating-label">Technique</div>\n<div class="rating-circles rating-max6"><div class="',''
    ).replace(
        '<div class="tour-rating">\n<div class="tour-rating-label">Fitness</div>\n<div class="rating-circles rating-max6"><div class="','').replace('"></div></div>\n</div>',"")
    rating_even_shorter = rating_all_short.replace('[','').replace(']','')
    rating_list = rating_even_shorter.split(", ")

    for i in range(0,(len(rating_list))):
        if i == 0:
            technique.append(rating_list[i]) # add first item in technique
        elif i % 2 == 0:
            technique.append(rating_list[i]) # then every second item as well
        else:
            fitness.append(rating_list[i]) # the other appended to fitness list
    return technique, fitness

In [30]:
def create_scraped_df(region, n_pages, save_as_csv = True):
    title, difficulty, sport, length, time_list, climb, minmax, total_title, rating = scraping_region(
        region, n_pages)
    id_df = extracting_id(total_title)
    technique, fitness = extracting_rating(rating)
    activities_df_unclean = pd.DataFrame(
    {'title': title,
     'difficulty': difficulty,
     'sport': sport,
     'length': length,
     'time': time_list,
     'climb': climb,
     'minmax': minmax,
     'technique': technique,
     'fitness': fitness
    })
    activities_df_unclean['ID']= id_df.iloc[:,5].copy() # add the ID columns from an earlier DF
    
    if save_as_csv == True:
        activities_df_unclean.to_csv(f"../data/unclean_activities_{region}.csv", index=False)
        
    return activities_df_unclean

Let's test it with different regions

In [37]:
region2 = "wallis"
n_pages2 = 2
#create_scraped_df(region2, n_pages2, save_as_csv = True)
#create_scraped_df('bern-region', n_pages2, save_as_csv = True)

scraped page 1
scraped page 2
number of tours collected: 40


,title,difficulty,sport,length,time,climb,minmax,technique,fitness,ID
0,[Zur Hohwacht im Oberaargau],[medium],[Hiking],13.19km,03:40h,460hm,539 - 779m,rating-2,rating-3,114208
1,[Lötschberger-Nordrampe],[medium],[Hiking],17.25km,04:50h,238hm,"779 - 1,205m",rating-3,rating-3,744928
2,[Wildhornhütte - Niesehore - Wildhornhütte],[medium],[Hiking],5.62km,03:00h,530hm,"2,257 - 2,774m",rating-3,rating-3,131695
3,[Diemtigtaler Hausweg «Mittleres Tal»],[easy],[Thematic route],10.2km,03:15h,434hm,"814 - 1,128m",rating-1,rating-2,295433
4,[Simmeflueh Klettersteig - Wimmis],[easy],[Via ferrata],2.09km,03:00h,767hm,"626 - 1,393m",rating-,rating-0,114228
5,[Rundtour über den Rüwlispass - Hornberg ins S...,[medium],[Cycling],31.82km,03:25h,950hm,"934 - 1,836m",rating-2,rating-4,700232
6,[Tour de Gruyère],[difficult],[Racing],79.88km,03:30h,"1,269hm","691 - 1,634m",rating-2,rating-5,131760
7,[Lenk - Hahnenmoospass - Geils - Bergläger - C...,[medium],[Cycling],24.69km,04:00h,"1,381hm","1,065 - 1,951m",rating-1,rating-3,691504
8,[Ins malerische Üschenetal],[medium],[Hiking],7.6km,02:15h,150hm,"1,174 - 1,795m",rating-3,rating-2,51086
9,[Bahnhof-Bühl-Niedermatti],[easy],[Hiking],3.56km,00:55h,34hm,"1,164 - 1,198m",rating-2,rating-2,744924


# 2  Data Cleaning

As mentioned above, the dataframe still needs to be cleaned up. 

### 2.0 Read in the "uncleaned" DF

In [40]:
region = 'wallis'
activities_df_raw = pd.read_csv(f"../data/unclean_activities_{region}.csv")
activities_df = activities_df_raw.copy()
activities_df

,title,difficulty,sport,length,time,climb,minmax,technique,fitness,ID
0,['Gemsweg'],['medium'],['Hiking'],7.87km,02:30h,106hm,"1,785 - 2,339m",rating-2,rating-3,601878
1,['Klettersteig Moiry'],['difficult'],['Via ferrata'],2.52km,02:30h,480hm,"2,108 - 2,311m",rating-,rating-5,92277
2,['Tour du Grand Chavalard'],['medium'],['Hiking'],15.74km,05:00h,895hm,"1,831 - 2,454m",rating-3,rating-4,175623
3,['Tärbiner Rundwanderweg'],['medium'],['Winter hiking'],10.17km,02:55h,364hm,"1,351 - 1,682m",rating-2,rating-3,908550
4,['Valais Vélo Tour 2: Crans-Montana - Leukerbad'],[],['Cycling'],40.97km,03:21h,"1,298hm","540 - 1,804m",rating-,rating-0,179696
5,['Panorama-Trail'],['medium'],['Snowshoe'],3.41km,01:30h,142hm,"1,982 - 2,121m",rating-3,rating-2,908541
6,['Col de Cou (The Brazilian)'],['medium'],['Mountainbike'],24.75km,02:59h,492hm,"503 - 2,522m",rating-4,rating-4,63632
7,['Grächen Loop'],['easy'],['Mountainbike'],10.42km,01:32h,330hm,"1,493 - 1,781m",rating-1,rating-2,310001
8,['Von Savoleyres zum Bergsee Lac des Vaux'],['medium'],['Hiking'],14.14km,04:30h,381hm,"1,521 - 2,554m",rating-3,rating-4,167055
9,['Bhutanbrücke'],['easy'],['Hiking'],10.48km,03:15h,335hm,"622 - 1,004m",rating-2,rating-2,185705


### 2.1 Clean Categorical Data
- Convert columns to strings
- Remove brackets that are in the first 3 columnns
- Remove extra whitespace

In [41]:
# Converting the text columns to strings for easier handling later
activities_df[['title', 'difficulty', 'sport']] = activities_df[['title', 'difficulty', 'sport']].astype('str')

# Removing the brackets
activities_df['title'] = activities_df['title'].str.strip('[]')
activities_df['difficulty'] = activities_df['difficulty'].str.strip('[]')
activities_df['sport'] = activities_df['sport'].str.strip('[]')

# Remove extra whitespace
activities_df['title'] = pd.Series(activities_df['title']).str.replace("'", '')
activities_df['difficulty'] = pd.Series(activities_df['difficulty']).str.replace("'", '')
activities_df['sport'] = pd.Series(activities_df['sport']).str.replace("'", '')

# Remove units (km, m, h, hm, m)
activities_df['length'] = pd.Series(activities_df['length']).str.replace("km", '')
activities_df['time'] = pd.Series(activities_df['time']).str.replace("h", '')
activities_df['climb'] = pd.Series(activities_df['climb']).str.replace("hm", '')
activities_df['minmax'] = pd.Series(activities_df['minmax']).str.replace("m", '')

# Seperate the minmax altitude into a min column and a max column
activities_df[['min','max']] = activities_df['minmax'].str.split("-",expand=True)
activities_df = activities_df.drop(columns=['minmax'])

# Organize the dataframe columns
activities_df = activities_df[['ID','title', 'difficulty', 'sport', 'length', 
                               'time', 'climb', 'min', 'max', 'technique', 'fitness']]

# Add ordering of the difficulty
activities_df['difficulty'] = pd.Series(activities_df['difficulty']).str.replace("easy", '1 - easy')
activities_df['difficulty'] = pd.Series(activities_df['difficulty']).str.replace("medium", '2 - medium')
activities_df['difficulty'] = pd.Series(activities_df['difficulty']).str.replace("difficult", '3 - difficult')
activities_df['difficulty'].replace(r'^\s*$', '0 - no rating', regex=True, inplace=True)

In [42]:
activities_df

,ID,title,difficulty,sport,length,time,climb,min,max,technique,fitness
0,601878,Gemsweg,2 - medium,Hiking,7.87,02:30,106,"1,785","2,339",rating-2,rating-3
1,92277,Klettersteig Moiry,3 - difficult,Via ferrata,2.52,02:30,480,"2,108","2,311",rating-,rating-5
2,175623,Tour du Grand Chavalard,2 - medium,Hiking,15.74,05:00,895,"1,831","2,454",rating-3,rating-4
3,908550,Tärbiner Rundwanderweg,2 - medium,Winter hiking,10.17,02:55,364,"1,351","1,682",rating-2,rating-3
4,179696,Valais Vélo Tour 2: Crans-Montana - Leukerbad,0 - no rating,Cycling,40.97,03:21,"1,298",540,"1,804",rating-,rating-0
5,908541,Panorama-Trail,2 - medium,Snowshoe,3.41,01:30,142,"1,982","2,121",rating-3,rating-2
6,63632,Col de Cou (The Brazilian),2 - medium,Mountainbike,24.75,02:59,492,503,"2,522",rating-4,rating-4
7,310001,Grächen Loop,1 - easy,Mountainbike,10.42,01:32,330,"1,493","1,781",rating-1,rating-2
8,167055,Von Savoleyres zum Bergsee Lac des Vaux,2 - medium,Hiking,14.14,04:30,381,"1,521","2,554",rating-3,rating-4
9,185705,Bhutanbrücke,1 - easy,Hiking,10.48,03:15,335,622,"1,004",rating-2,rating-2


### 2.2 Clean Numerical Data

In order to normalize numerical data for analysis, we must preform the following steps:

-  Replace missing numerical data with 0
-  Remove commas
-  Remove "-"
-  Remove unnecessary words ("rating-")


In [43]:
activities_df['length'] = (activities_df['length']).astype(float)

activities_df['climb'] = pd.Series(activities_df['climb']).str.replace(",", '') 
activities_df['climb'] = pd.Series(activities_df['climb']).str.replace("-", '0') 
activities_df['climb'] = (activities_df['climb']).astype(int)

activities_df['min'] = pd.Series(activities_df['min']).str.replace(",", '') 
activities_df['min'] = (activities_df['min']).astype(int)
activities_df['max'] = pd.Series(activities_df['max']).str.replace(",", '') 
activities_df['max'] = (activities_df['max']).astype(int)

activities_df['technique'] = pd.Series(activities_df['technique']).str.replace("rating-", '')
activities_df['technique'] = pd.to_numeric(activities_df['technique'], errors='coerce')
activities_df['technique'] = pd.Series(activities_df['technique']).replace(np.nan, 0, regex = True)
activities_df['technique'] = pd.Series(activities_df['technique']).astype('int')

activities_df['fitness'] = pd.Series(activities_df['fitness']).str.replace("rating-", '')
activities_df['fitness'] = pd.to_numeric(activities_df['fitness'], errors='coerce')
activities_df['fitness'] = pd.Series(activities_df['fitness']).replace(np.nan, 0, regex = True)
activities_df['fitness'] = activities_df['fitness'].astype('int')

### 2.3 Clean Time Data
Some of the hikes can register times >24 hours, and this causes Pandas to throw an error. This is handled by using Try/Except

In [44]:
activities_df

,ID,title,difficulty,sport,length,time,climb,min,max,technique,fitness
0,601878,Gemsweg,2 - medium,Hiking,7.87,02:30,106,1785,2339,2,3
1,92277,Klettersteig Moiry,3 - difficult,Via ferrata,2.52,02:30,480,2108,2311,0,5
2,175623,Tour du Grand Chavalard,2 - medium,Hiking,15.74,05:00,895,1831,2454,3,4
3,908550,Tärbiner Rundwanderweg,2 - medium,Winter hiking,10.17,02:55,364,1351,1682,2,3
4,179696,Valais Vélo Tour 2: Crans-Montana - Leukerbad,0 - no rating,Cycling,40.97,03:21,1298,540,1804,0,0
5,908541,Panorama-Trail,2 - medium,Snowshoe,3.41,01:30,142,1982,2121,3,2
6,63632,Col de Cou (The Brazilian),2 - medium,Mountainbike,24.75,02:59,492,503,2522,4,4
7,310001,Grächen Loop,1 - easy,Mountainbike,10.42,01:32,330,1493,1781,1,2
8,167055,Von Savoleyres zum Bergsee Lac des Vaux,2 - medium,Hiking,14.14,04:30,381,1521,2554,3,4
9,185705,Bhutanbrücke,1 - easy,Hiking,10.48,03:15,335,622,1004,2,2


In [ ]:
# convert times to time objects
try:
    activities_df['time'] = activities_df['time'].dt.strftime('%H:%M')
except ValueError:
    pass

### 2.4 Final check of DataFrame
Double check for null values. It looks pretty good!  Now it's time to start analysis. 

In [45]:
activities_df

,ID,title,difficulty,sport,length,time,climb,min,max,technique,fitness
0,601878,Gemsweg,2 - medium,Hiking,7.87,02:30,106,1785,2339,2,3
1,92277,Klettersteig Moiry,3 - difficult,Via ferrata,2.52,02:30,480,2108,2311,0,5
2,175623,Tour du Grand Chavalard,2 - medium,Hiking,15.74,05:00,895,1831,2454,3,4
3,908550,Tärbiner Rundwanderweg,2 - medium,Winter hiking,10.17,02:55,364,1351,1682,2,3
4,179696,Valais Vélo Tour 2: Crans-Montana - Leukerbad,0 - no rating,Cycling,40.97,03:21,1298,540,1804,0,0
5,908541,Panorama-Trail,2 - medium,Snowshoe,3.41,01:30,142,1982,2121,3,2
6,63632,Col de Cou (The Brazilian),2 - medium,Mountainbike,24.75,02:59,492,503,2522,4,4
7,310001,Grächen Loop,1 - easy,Mountainbike,10.42,01:32,330,1493,1781,1,2
8,167055,Von Savoleyres zum Bergsee Lac des Vaux,2 - medium,Hiking,14.14,04:30,381,1521,2554,3,4
9,185705,Bhutanbrücke,1 - easy,Hiking,10.48,03:15,335,622,1004,2,2


In [46]:
activities_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ID          40 non-null     int64  
 1   title       40 non-null     object 
 2   difficulty  40 non-null     object 
 3   sport       40 non-null     object 
 4   length      40 non-null     float64
 5   time        40 non-null     object 
 6   climb       40 non-null     int64  
 7   min         40 non-null     int64  
 8   max         40 non-null     int64  
 9   technique   40 non-null     int64  
 10  fitness     40 non-null     int64  
dtypes: float64(1), int64(6), object(4)
memory usage: 3.6+ KB


### 2.5 Write CSV

Now that the data has been cleaned properly, let's save the dataframe to a CSV for future use and hash out the scraping code in 1.1. Otherwise if you rerun the Kernel too often, there is a risk of being blocked by the website.

In [ ]:
activities_df.to_csv("../data/activities.csv", index=False)

### 2.6 Functions for Data Cleaning

In [50]:
def clean_cat(activities_df):
    # Converting the text columns to strings for easier handling later
    activities_df[['title', 'difficulty', 'sport']] = activities_df[['title', 'difficulty', 'sport']].astype('str')

    # Removing the brackets
    activities_df['title'] = activities_df['title'].str.strip('[]')
    activities_df['difficulty'] = activities_df['difficulty'].str.strip('[]')
    activities_df['sport'] = activities_df['sport'].str.strip('[]')

    # Remove extra whitespace
    activities_df['title'] = pd.Series(activities_df['title']).str.replace("'", '')
    activities_df['difficulty'] = pd.Series(activities_df['difficulty']).str.replace("'", '')
    activities_df['sport'] = pd.Series(activities_df['sport']).str.replace("'", '')

    # Remove units (km, m, h, hm, m)
    activities_df['length'] = pd.Series(activities_df['length']).str.replace("km", '')
    activities_df['time'] = pd.Series(activities_df['time']).str.replace("h", '')
    activities_df['climb'] = pd.Series(activities_df['climb']).str.replace("hm", '')
    activities_df['minmax'] = pd.Series(activities_df['minmax']).str.replace("m", '')

    # Seperate the minmax altitude into a min column and a max column
    activities_df[['min','max']] = activities_df['minmax'].str.split("-",expand=True)
    activities_df = activities_df.drop(columns=['minmax'])

    # Organize the dataframe columns
    activities_df = activities_df[['ID','title', 'difficulty', 'sport', 'length', 
                                   'time', 'climb', 'min', 'max', 'technique', 'fitness']]

    # Add ordering of the difficulty
    activities_df['difficulty'] = pd.Series(activities_df['difficulty']).str.replace("easy", '1 - easy')
    activities_df['difficulty'] = pd.Series(activities_df['difficulty']).str.replace("medium", '2 - medium')
    activities_df['difficulty'] = pd.Series(activities_df['difficulty']).str.replace("difficult", '3 - difficult')
    activities_df['difficulty'].replace(r'^\s*$', '0 - no rating', regex=True, inplace=True)
    return activities_df

In [51]:
def clean_num(activities_df):
    activities_df['length'] = (activities_df['length']).astype(float)

    activities_df['climb'] = pd.Series(activities_df['climb']).str.replace(",", '') 
    activities_df['climb'] = pd.Series(activities_df['climb']).str.replace("-", '0') 
    activities_df['climb'] = (activities_df['climb']).astype(int)

    activities_df['min'] = pd.Series(activities_df['min']).str.replace(",", '') 
    activities_df['min'] = (activities_df['min']).astype(int)
    activities_df['max'] = pd.Series(activities_df['max']).str.replace(",", '') 
    activities_df['max'] = (activities_df['max']).astype(int)

    activities_df['technique'] = pd.Series(activities_df['technique']).str.replace("rating-", '')
    activities_df['technique'] = pd.to_numeric(activities_df['technique'], errors='coerce')
    activities_df['technique'] = pd.Series(activities_df['technique']).replace(np.nan, 0, regex = True)
    activities_df['technique'] = pd.Series(activities_df['technique']).astype('int')

    activities_df['fitness'] = pd.Series(activities_df['fitness']).str.replace("rating-", '')
    activities_df['fitness'] = pd.to_numeric(activities_df['fitness'], errors='coerce')
    activities_df['fitness'] = pd.Series(activities_df['fitness']).replace(np.nan, 0, regex = True)
    activities_df['fitness'] = activities_df['fitness'].astype('int')
    return activities_df

In [52]:
def clean_time(activities_df):
    try:
        activities_df['time'] = activities_df['time'].dt.strftime('%H:%M')
    except ValueError:
        pass
    return activities_df

In [61]:
def clean_df(activities_df, save_as_csv = True):
    activities_df = clean_cat(activities_df)
    activities_df = clean_num(activities_df)
    #clean_time(activities_df)
    
    if save_as_csv == True:
        activities_df.to_csv(f"../data/activities_{region}.csv", index=False)
        
    return activities_df

In [62]:
def preprocess(region, n_pages, save_as_csv = True):
    if save_as_csv == True:
        save = True
    else:
        save = False
    activities_df_unclean = create_scraped_df(region, n_pages, save_as_csv = save)
    activities_df = clean_df(activities_df_unclean, save_as_csv = save)
        
    return activities_df

In [63]:
region3 = 'genferseegebiet-waadtland'
res3 = preprocess(region3,n_pages2,save_as_csv = True)

scraped page 1
scraped page 2
number of tours collected: 40


In [ ]:
df_test_unclear = pd.read_csv(file_path)
df = df_raw.copy()

In [ ]:
clean_df()

In [60]:
activities_df_unclean

NameError: name 'activities_df_unclean' is not defined

In [ ]:
clean_df(activities_df_unclean, save_as_csv = save)

# 3  Initial Data Analysis

Before moving onto combining weather data with hiking trails, lets do some preliminary data analysis to see what we are working with.

In [ ]:
plt.style.use('fivethirtyeight')
rcParams['figure.figsize'] = 10, 8

### 3.1 Data Exploration 

We will begin the data analysis by data exploration. By creating different data frames and removing activities with little data to improve visualizations we can get a better idea of what types of trails we are working with. 

Here we can see that cycling, racing, and long-distance walking are much longer than the other types of trails. Another helpful tool this DataFrame provides is that we can now view the least used trail types in the dataset. These are going to be removed for the next few analysis steps so it's easier to look at the interesting trail statistics.

In [ ]:
# Creating a df for "Number of entries by sport type" + average tour length
sporttype = activities_df.groupby('sport') \
       .agg({'sport':'count', 'length':'mean'}) \
       .rename(columns={'sport':'count','length':'average_length'}) \
       .reset_index() \
       .sort_values(by=['average_length'])
sporttype

### 3.2 Focus on the most relevant activities
Remove the activities, which have less than 10 entries.

In [ ]:
analysis = activities_df.groupby("sport").filter(lambda x: len(x) > 10)
analysis.groupby(['sport']).size()

### 3.3 Bar Chart Visualization
Now that the activities with little entries are removed, a barchart of the sport types is much more readable. We can see that the most popular activity on our list of trails is hiking with more than 250 entries.

In [ ]:
ax = analysis['sport'].value_counts().plot(kind='bar')
plt.title('Frequency of Activity types')
plt.xlabel('Type of Activity')
plt.ylabel('Count of Activity');

How are the picture look like by level of difficulty?

In [ ]:
analysis['difficulty'].value_counts().plot(kind='bar')
plt.title('Number of Activities by Difficulty')
plt.xlabel('Difficulty')
plt.ylabel('Count of Activity');

### 3.4 Scatter Plot Visualization

The scatter plot paints an interesting picture of the trail data. First, the cycling trails are both the longest and the ones with most height meter difference out of the bunch. And second, it looks like all of the hiking trails fall between 2 and 20km (except for one). Perhaps this is because long distance hiking trails are considered to be "Pilgrimages"?

In [ ]:
sns.scatterplot(    
    data=analysis,
    x="length", y="climb", hue='sport',
).set_title('Length and Distance By Sport Type');

In [ ]:
# mean length for difficulty:
activities_df.groupby('difficulty').mean()

Now let's explore the data using only hiking trail data.

# 4 Analysis: Hiking Trails 

The primary interest of this analysis is hiking trail data. The following analysis will compare only hiking trails and drop the rest of the sport types.

In [ ]:
hiking = activities_df[activities_df.sport == "Hiking"]
hiking.shape

### 4.1 Scatter plot: Length and Distance of each Activity By Difficulty

In [ ]:
sns.scatterplot(    
    data=hiking,
    x="length", y="climb", hue='difficulty',
).set_title('Length and Distance By Difficulty');

Seems we have one "outlier" which is more than 200km long. Let's remove it to get a better view on the majority of the trails.

In [ ]:
sns.scatterplot(    
    data=hiking[hiking.length < 200],
    x="length", y="climb", hue='difficulty',
).set_title('Length and Distance By Difficulty');

The easier trails seem to be shorter in length and with lower elevation gain. Let's see if there is a correlation.

### 4.2 Line Plot: Comparison of average Trail Length vs. Fitness Score

The following dataframe uses the fitness scores and compares them to the length of each trail. How correlated is the trail length to the fitness score? A new columns is added for average length. 

In [ ]:
# calculating the count by Fitness level and the average length 
fitness_length_check = hiking.groupby('fitness') \
       .agg({'fitness':'count', 'length':'mean'}) \
       .rename(columns={'fitness':'count','length':'average_length'}) \
       .reset_index()
fitness_length_check

The routes with no fitness rating seems quite long. Probably related to the outlier previously identified.
For further analysis let's remove the trails without any difficulty rating and check if there is a visual clue.

In [ ]:
fitness_length = fitness_length_check[fitness_length_check['fitness'] != 0]
plt.plot(fitness_length["fitness"], fitness_length["average_length"])
plt.ylabel('average_length')
plt.xlabel("rating_fitness")
plt.title('Hiking: Average Length by Fitness Level Rating')
plt.show()

There seems to be a relationship indeed between fitness score and length of the hike.

### 4.3 Boxplots: Difficulty vs. Elevation Gain and Length

The following boxplots show how the trail data is distributed by difficulty catagory compared to both elevation gain and length. It's pretty clear that the relationship between elevation gain and difficulty is more meaningful than the relationship between distance and difficulty. My legs agree! 

In [ ]:
my_pal = {"1 - easy": "lightgreen", "2 - medium": "gold", "3 - difficult":"salmon", 
          "0 - no rating":"blanchedalmond"}
ax = sns.boxplot(x="difficulty", y="climb", data=hiking, 
                 order=["1 - easy", "2 - medium", "3 - difficult", "0 - no rating"],
                palette=my_pal) \
.set_title('Relationship between Trail Difficulty Rating and Elevation Gain')\
#.set_style("white")

# https://matplotlib.org/2.0.2/examples/color/named_colors.html

In [ ]:
ax = sns.boxplot(x="difficulty", y="length",showfliers = False, data=hiking, 
                 order=["1 - easy", "2 - medium", "3 - difficult", "0 - no rating"],
                palette=my_pal) \
.set_title('Relationship between Trail Difficulty Rating and Distance')

### 3.6 Correlation Matrix
Let's see what the mathematical correlations are between the four parameters: climb, length, fitness score, technique score.

In [ ]:
corr_matrix_df = hiking.drop(columns = ['min','max']).copy()
corr_matrix = corr_matrix_df.corr()
plt.figure(figsize=(14,8))
sns.heatmap(corr_matrix, annot = True,cmap = 'Blues');

'Fitness' and 'Technique' have the highest correlation among all measures.
What about the correlation with the 'Difficulty Level'?
We therefore convert the 'easy', 'medium', 'difficult' in numeric values and build a correlation matrix.

In [ ]:
corr_matrix_df['difficulty_level'] = corr_matrix_df['difficulty']
corr_matrix_df.loc[corr_matrix_df['difficulty'] == 'easy', ['difficulty_level']] = 1
corr_matrix_df.loc[corr_matrix_df['difficulty'] == 'medium', ['difficulty_level']] = 2
corr_matrix_df.loc[corr_matrix_df['difficulty'] == 'difficult', ['difficulty_level']] = 3
corr_matrix_df.loc[corr_matrix_df['difficulty'] == '', ['difficulty_level']] = 0
# the column 'difficulty_level' will return object type which we need to convert to integers.
corr_matrix_df['difficulty_level']= corr_matrix_df['difficulty_level'].astype(str).astype(int)
#or corr_matrix_df2 = pd.to_numeric(corr_matrix_df['difficulty_level']).astype(int)
corr_matrix_df2 = corr_matrix_df.copy()
corr_matrix_df2

In [ ]:
corr_matrix_level = corr_matrix_df2.corr()
plt.figure(figsize=(14,8))
sns.heatmap(corr_matrix_level, annot = True,cmap = 'Blues');

As we can see, the highest correlation is between the fitness level and the difficulty level, followed by the technique and the difficulty level. This was not visible in the box plot graphs above.

# 5 Download Trail GPS Data

The next part of this notebook deals with parsing HTML tags again, however this time around we will be gathering the GPS data needed to create the snowfall alert. The GPS data for each trail is actually a list of GPS points that comprise the length of the trail, from start to summit. However, we will only be using the first GPS point in our algorithm, which is the trail-head. 

The Bergfex website embedds their GPS data into downloadable files, which are meant to be uploaded to a GPS device for navigation purposes. As such, the following code will automatically download each GPS file into a folder. Importantly, a sleep timer is added to replicate how a human may download these files, if the sleep timer is not added, the website will become suspicious and prevent us from downloading more files. 

The type of GPS file that is being downloaded is a "GPX" or "GPS Exchange Format" file, which is a special type of XML file that can be used to describe waypoints. For testing purposes, these GPX files were uploaded into Google Earth, and it correctly displayed the location. 

- https://www.bergfex.com/downloads/gps/?type=&id=271548&fileType=kml


### 5.1 Download Batches of GPX Files

This loop uses the hike id's to navigate to each of the hike's pages. It then downloads the GPS file to the folder "gpx_files" and saves it as a filename "Unique hike id.gpx". To prevent being blocked from the page, we download the dat in small batches for all 400 activities.

In [ ]:
test_id = corr_matrix_df.iloc[20:30,0]

In [ ]:
# create a small subset of data for testing
test_id = corr_matrix_df.iloc[20:30,0]
for hike_id in test_id: 
    url_id = f"https://www.bergfex.com/downloads/gps/?type=&id={hike_id};fileType=kml"
    print(url_id)
    r = requests.get(url_id, allow_redirects=True)
    time.sleep(5) 
    write_link = ('../gpx_files/'+ hike_id + ".gpx")
    # Replace kml files with your filename
    with open(write_link, 'wb') as f:
        f.write(r.content)

###  5.2 Extract Coordinates from GPX files

Now that the GPX files have been downloaded to a local file, we can loop through them and extract the important information, which is the latitude and the longitude of the trail start.

In [ ]:
list_of_trails_list = []
list_of_files = glob.glob('*gpx')  
for hike_id in test_id: 
    trail_gps_list = []
    gpx = gpxpy.parse(open('../gpx_files/'+hike_id+'.gpx', 'r'))
    trail_gps_list.append(hike_id)
    for track in gpx.tracks: 
        trail_name = track.name
        trail_gps_list.append(trail_name)
        for segment in track.segments: 
            for point in segment.points: 
              lat = point.latitude
              trail_gps_list.append(lat)
              lon = point.longitude
              trail_gps_list.append(lon) 
              break
        list_of_trails_list.append(trail_gps_list)

print(list_of_trails_list)

### 5.3  Create a Dataframe and Save as CSV

Lets take a look at our data now that it has been extracted. It looks like everything has is working as intended. Each trail has a unique ID and latitude and longitude. This data can now be saved to a CSV file which will be used for the snow alert in the next notebook. 

In [ ]:
trail_coords_df = pd.DataFrame(list_of_trails_list)
# name the columns
trail_coords_df.columns=['trail_id', 'name', 'lat', 'lon']
trail_coords_df.set_index('trail_id')

In [ ]:
trail_coords_df.to_csv('../data/trail_coords_df.csv', index=False)
read= pd.read_csv('../data/trail_coords_df.csv')
read

# CONTINUE IN NOTEBOOK II